In [8]:
import csv
import os
from datetime import datetime

def load_items():
    items = {}
    with open("items.csv", mode="r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            items[int(row["ItemID"])] = {"Product": row["Product"], "Price": float(row["Price"])}
    return items

def load_save_for_later(name):
    saved = []
    if os.path.exists("save_for_later.csv"):
        with open("save_for_later.csv", mode="r") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row["Name"] == name:
                    saved.append({"ItemID": int(row["ItemID"]), "Qty": int(row["Qty"])})
    return saved

In [9]:
def display_items(items):
    print("Available Items:")
    print("ID | Product | Price")
    for i, d in items.items():
        print(f"{i} | {d['Product']} | {d['Price']}")

def display_saved_items(saved, items):
    if saved:
        print("\nSaved For Later:")
        print("ID | Product | Qty")
        for entry in saved:
            pid = entry["ItemID"]
            qty = entry["Qty"]
            print(f"{pid} | {items[pid]['Product']} | {qty}")
    else:
        print("\nNo items in Save for Later.")

In [10]:
def process_user_input(user_input, cart, save_later, items, saved_items):
    parts = user_input.strip().split(',')
    if len(parts) >= 2:
        try:
            pid = int(parts[0])
            qty = int(parts[1])
            if pid not in items:
                print(f"Item ID {pid} not found. Please enter a valid ID.")
                return

            if len(parts) == 3 and parts[2].lower() == "save":
                save_later[pid] = qty
                return

            used_from_saved = 0
            for s in saved_items:
                if s["ItemID"] == pid:
                    used_from_saved = min(s["Qty"], qty)
                    s["Qty"] -= used_from_saved
                    if s["Qty"] <= 0:
                        saved_items.remove(s)
                    break

            remaining_from_items = qty - used_from_saved
            total_qty = used_from_saved + remaining_from_items

            cart[pid] = total_qty

        except ValueError:
            print("Invalid input format. Use: itemID,qty or itemID,qty,save")

In [11]:
def show_summary(cart, save_later, items):
    print("\nCart Summary:")
    total = 0
    print("ID | Product | Qty | Price | Subtotal")
    for pid, qty in cart.items():
        name = items[pid]['Product']
        price = items[pid]['Price']
        subtotal = qty * price
        total += subtotal
        print(f"{pid} | {name} | {qty} | {price} | {subtotal}")
    print(f"Total: ₹{total}")
    
    if save_later:
        print("\nSaved For Later:")
        print("ID | Product | Qty")
        for pid, qty in save_later.items():
            print(f"{pid} | {items[pid]['Product']} | {qty}")
    else:
        print("\nNo items in Save for Later.")
    return total

In [12]:
def get_next_transaction_id():
    if not os.path.exists("transactions.csv"):
        return 1
    with open("transactions.csv", mode="r") as f:
        reader = csv.DictReader(f)
        ids = [int(row["TransactionID"]) for row in reader]
        return max(ids) + 1 if ids else 1

def save_transaction(tid, name, cart, total):
    with open("transactions.csv", mode="a", newline='') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(["TransactionID", "Name", "Items", "Total"])
        items_str = ",".join([f"{pid}:{qty}" for pid, qty in cart.items()])
        writer.writerow([tid, name, items_str, total])

def save_save_for_later(name, save_later):
    updated_rows = []

    if os.path.exists("save_for_later.csv"):
        with open("save_for_later.csv", mode="r") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row["Name"] != name:
                    updated_rows.append(row)

    for pid, qty in save_later.items():
        updated_rows.append({"Name": name, "ItemID": str(pid), "Qty": str(qty)})

    with open("save_for_later.csv", mode="w", newline='') as f:
        fieldnames = ["Name", "ItemID", "Qty"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in updated_rows:
            writer.writerow(row)


def save_feedback(tid, feedback):
    with open("feedback.csv", mode="a", newline='') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(["TransactionID", "Feedback"])
        writer.writerow([tid, feedback])

In [13]:
def grocery_app():
    items = load_items()
    name = input("Enter your name: ").strip().lower()
    saved_items = load_save_for_later(name)
    
    cart = {}
    save_later = {}

    display_items(items)
    display_saved_items(saved_items, items)

    while True:
        user_input = input("\nEnter itemID,qty OR itemID,qty,save OR done: ").strip().lower()
        if user_input == "done":
            break
        process_user_input(user_input, cart, save_later, items, saved_items)
        show_summary(cart, save_later, items)

    total = show_summary(cart, save_later, items)

    confirm = input("\nDo you want to proceed with payment? (y/n): ").strip().lower()
    if confirm == "y":
        tid = get_next_transaction_id()
        save_transaction(tid, name, cart, total)
        save_save_for_later(name, save_later)
        print("\nPayment Successful!\n")

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print("Invoice")
        print(f"Transaction ID: {tid}")
        print(f"Name         : {name}")
        print(f"Date & Time  : {timestamp}")
        print("Items:")
        for pid, qty in cart.items():
            pname = items[pid]['Product']
            price = items[pid]['Price']
            subtotal = qty * price
            print(f"- {pname} (x{qty}) @ ₹{price} = ₹{subtotal}")
        print(f"Total: ₹{total}")

        fb_choice = input("\nWould you like to leave feedback? (y/n): ").strip().lower()
        if fb_choice == "y":
            fb = input("Enter your feedback: ").strip()
            if fb:
                save_feedback(tid, fb)
                print("Feedback saved.")
            else:
                print("No feedback entered.")
        else:
            print("No feedback given.")
    else:
        print("Order not placed.")

In [14]:
grocery_app()

Available Items:
ID | Product | Price
1 | Potato | 10.0
2 | Tomato | 12.0
3 | Bread | 25.0
4 | Butter | 45.0
5 | Milk | 20.0
6 | Onion | 12.0
7 | Apple | 25.0
8 | Sugar | 40.0
9 | Rice | 60.0
10 | Paneer | 55.0

Saved For Later:
ID | Product | Qty
2 | Tomato | 3

Cart Summary:
ID | Product | Qty | Price | Subtotal
2 | Tomato | 3 | 12.0 | 36.0
Total: ₹36.0

No items in Save for Later.

Cart Summary:
ID | Product | Qty | Price | Subtotal
2 | Tomato | 3 | 12.0 | 36.0
Total: ₹36.0

No items in Save for Later.

Cart Summary:
ID | Product | Qty | Price | Subtotal
2 | Tomato | 3 | 12.0 | 36.0
Total: ₹36.0

No items in Save for Later.

Payment Successful!

Invoice
Transaction ID: 11
Name         : r
Date & Time  : 2025-07-07 10:34:47
Items:
- Tomato (x3) @ ₹12.0 = ₹36.0
Total: ₹36.0
Feedback saved.
